In [11]:
import pandas as pd
import numpy as np
import os
from wrangle import acquire, prep, wrangle_zillow
import warnings
warnings.filterwarnings("ignore")

In [2]:
# function call to acquire df from SQL and store on this computer as a csv before cleaning
df = acquire()

In [3]:
# I need to rename these columns so they're easier to type out
df = df.rename(columns={'bedroomcnt': 'beds', 'bathroomcnt': 'baths', 'calculatedfinishedsquarefeet': 'square_feet', 'taxvaluedollarcnt' : 'value', 'yearbuilt': 'year_built', 'taxamount': 'tax_amount', 'fips': 'federal_processing_code'})
# taking a quick look at the info
df

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [4]:
df.beds.value_counts(dropna=False, ascending=True)

beds
25.0         1
16.0         2
18.0         3
15.0         6
14.0         7
NaN         11
12.0        12
13.0        16
11.0        34
10.0       121
9.0        291
8.0       1107
7.0       4807
0.0      13187
1.0      23166
6.0      25166
5.0     150866
2.0     335473
4.0     634289
3.0     964298
Name: count, dtype: int64

In [5]:
df[df.beds==0]

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
5,0.0,0.0,NaN,124.0,NaN,NaN,6037.0
...,...,...,...,...,...,...,...
2152823,0.0,0.0,NaN,2568893.0,NaN,27309.30,6059.0
2152844,0.0,0.0,NaN,92679.0,NaN,1090.16,6111.0
2152854,0.0,0.0,NaN,1198476.0,NaN,NaN,6037.0
2152857,0.0,0.0,NaN,1087111.0,NaN,19313.08,6059.0


In [6]:
# Then replace the values of 0 in beds and bath will null since it's the same thing
df['beds'] = df['beds'].replace(0.0, np.nan)
df['baths'] = df['baths'].replace(0.0, np.nan)
# the federal processing code is presented with no null values and does not need to have a decimal
df['federal_processing_code'] = df['federal_processing_code'].astype(int)
df

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
0,NaN,NaN,NaN,27516.0,NaN,NaN,6037
1,NaN,NaN,NaN,10.0,NaN,NaN,6037
2,NaN,NaN,NaN,10.0,NaN,NaN,6037
3,NaN,NaN,NaN,2108.0,NaN,174.21,6037
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037
...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059
2152860,NaN,NaN,NaN,208057.0,NaN,5783.88,6059
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059


In [7]:
df.isnull().sum()

beds                       13198
baths                      13038
square_feet                 8484
value                        493
year_built                  9337
tax_amount                  4442
federal_processing_code        0
dtype: int64

In [12]:
# I'm gonna go ahead and drop the null values since collectively they make up about 1/8th of a percent and I don't feel that's significant enough to make a change in the dataframe.
df = df.dropna()
# I'm also going to convert most of my columns into integers since I don't need to know there's 4.0 bedrooms.
df['beds'] = df['beds'].astype(int)
df['year_built'] = df['year_built'].astype(int)
df['square_feet'] = df['square_feet'].astype(int)
df['value'] = df['value'].astype(int)
df

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
4,4,2.0,3633,296425,2005,6941.39,6037
6,3,4.0,1620,847770,2011,10244.94,6037
7,3,2.0,2077,646760,1926,7924.68,6037
18,3,1.0,1244,169471,1950,2532.88,6037
19,3,2.0,1300,233266,1950,3110.99,6037
...,...,...,...,...,...,...,...
2152856,4,4.0,4375,422400,2015,13877.56,6037
2152858,4,3.0,2262,960756,2015,13494.52,6059
2152859,4,4.5,3127,536061,2014,6244.16,6059
2152861,3,2.5,1974,424353,2015,5302.70,6059


In [13]:
# Code to be moved to wrangle function for preparation part
# df = df.rename(columns={'bedroomcnt': 'beds', 'bathroomcnt': 'baths', 'calculatedfinishedsquarefeet': 'square_feet', 'taxvaluedollarcnt' : 'value', 'yearbuilt': 'year_built', 'taxamount': 'tax_amount', 'fips': 'federal_processing_code'})
# df['beds'] = df['beds'].replace(0.0, np.nan)
# df['baths'] = df['baths'].replace(0.0, np.nan)
# df['federal_processing_code'] = df['federal_processing_code'].astype(int)
# df = df.dropna()
# df['beds'] = df['beds'].astype(int)
# df['year_built'] = df['year_built'].astype(int)
# df['square_feet'] = df['square_feet'].astype(int)
# df['value'] = df['value'].astype(int)

In [9]:
# function call to test prep function
df = prep()
df

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
4,4,2.0,3633,296425,2005,6941.39,6037
6,3,4.0,1620,847770,2011,10244.94,6037
7,3,2.0,2077,646760,1926,7924.68,6037
18,3,1.0,1244,169471,1950,2532.88,6037
19,3,2.0,1300,233266,1950,3110.99,6037
...,...,...,...,...,...,...,...
2152856,4,4.0,4375,422400,2015,13877.56,6037
2152858,4,3.0,2262,960756,2015,13494.52,6059
2152859,4,4.5,3127,536061,2014,6244.16,6059
2152861,3,2.5,1974,424353,2015,5302.70,6059


In [10]:
# Wrangle function testing
new_df = wrangle_zillow()
new_df

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
4,4,2.0,3633,296425,2005,6941.39,6037
6,3,4.0,1620,847770,2011,10244.94,6037
7,3,2.0,2077,646760,1926,7924.68,6037
18,3,1.0,1244,169471,1950,2532.88,6037
19,3,2.0,1300,233266,1950,3110.99,6037
...,...,...,...,...,...,...,...
2152856,4,4.0,4375,422400,2015,13877.56,6037
2152858,4,3.0,2262,960756,2015,13494.52,6059
2152859,4,4.5,3127,536061,2014,6244.16,6059
2152861,3,2.5,1974,424353,2015,5302.70,6059


In [32]:
new_df.reset_index()

TypeError: 'bool' object is not callable

In [28]:
new_df.sort_values(by='year_built', ascending = False)

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
21548,4,4.5,4077,720627,2016,9845.52,6111
258457,4,4.0,4144,1027568,2016,11801.40,6037
1248832,4,2.0,2078,280000,2016,5569.86,6037
2077445,4,5.0,4050,679770,2016,8384.15,6037
613813,4,2.0,2078,239753,2016,4985.55,6037
...,...,...,...,...,...,...,...
2029622,4,3.0,3006,307192,1812,3894.16,6037
1104175,3,1.0,1136,67869,1807,909.47,6037
1823373,3,3.0,4765,401964,1801,5658.76,6037
1005349,4,3.0,3652,1178436,1801,15682.27,6037


In [29]:
new_df.sort_values(by='value', ascending = False)

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
530834,11,16.0,52503,90188462,1990,1078101.87,6037
950854,12,15.0,35046,88020448,2002,1052070.50,6037
1618393,7,20.0,28725,83196095,1938,994030.96,6037
2134692,5,8.0,22300,71067500,2014,824166.91,6037
430030,7,14.0,22396,70416358,2001,841059.04,6037
...,...,...,...,...,...,...,...
1574215,2,1.0,924,23,1949,113.50,6037
2002187,4,2.0,2378,22,2006,990.26,6037
1851761,3,3.0,1428,22,1954,6834.33,6037
1765721,2,1.0,943,22,1949,135.93,6037


In [31]:
new_df.sort_values(by='beds', ascending = False)

,beds,baths,square_feet,value,year_built,tax_amount,federal_processing_code
26485,25,20.0,11700,1608491,2010,19238.87,6037
799516,18,18.0,9309,2325000,1986,29283.87,6037
1758193,18,18.0,11134,2353308,1986,29165.39,6037
1866663,16,16.0,7121,975000,1987,11218.15,6037
650428,16,5.0,2584,543654,1922,6490.08,6037
...,...,...,...,...,...,...,...
950234,1,1.0,840,125439,1923,1563.67,6037
1497494,1,1.0,763,90918,1978,1060.56,6111
1793831,1,2.0,1578,533006,1962,6349.91,6037
273990,1,1.0,681,64330,1906,1061.31,6037
